In [114]:
import pandas as pd

file = "assets/sel03/A0701_SEL03_TB_AN_00_2025_06_P_EN.xlsx"

#list all sheets names in file
sheets = pd.ExcelFile(file).sheet_names
print(sheets)

['Cover page', 'Table 1', 'Table 2A', 'Table 2C', 'Table 2D', 'Table 3A', 'Table 3B', 'Table 3D', 'Table 3E', 'Table 4', 'Edp', 'Parameters']


In [115]:
def load_and_clean_table(file, sheet_name, col_indices, col_names, code_prefix="A.N"):
    # Load the table
    df = pd.read_excel(file, sheet_name=sheet_name)
    
    # Keep only selected columns
    df = df.iloc[:, col_indices]
    df.columns = col_names

    # Keep rows where "Code" starts with the given prefix
    df = df[df["Code"].fillna("").str.startswith(code_prefix)]

    # Ensure all year columns are numeric
    for year in col_names[1:]:
        df = df[pd.to_numeric(df[year], errors='coerce').notnull()]
    
    return df



def custom_reorder(code):
    parts = code.split(".")
    
    # Step 1: Remove index 11
    if len(parts) > 11:
        del parts[11]
    
    # Step 2: Move index -3 to -2
    if len(parts) >= 3:
        token = parts.pop(-3)
        parts.insert(-2, token)
    
    # Step 3: Move new index 11 to index -3
    if len(parts) > 11:
        token = parts.pop(11)
        parts.insert(-3, token)
    
    # Step 4: Move new index 11 to index -4
    if len(parts) > 11:
        token = parts.pop(11)
        parts.insert(-4, token)

    token = parts.pop(-2)
    parts.insert(-4, token)

    # swap index -4 with index -5
    if len(parts) > 5:
        parts[-4], parts[-5] = parts[-5], parts[-4]
    
    return ".".join(parts)

In [116]:
# Define column indices and names
columns_indices_table1 = [1, 4, 5, 6, 7]
columns_indices_table2 = [1, 3, 4, 5, 6]
columns_indices_table3 = [1, 3, 4, 5, 6]
columns_indices_table4 = [1, 5, 6, 7, 8]
column_names = ["Code", "2021", "2022", "2023", "2024"]

# Load and clean both tables
table1 = load_and_clean_table(file, "Table 1", columns_indices_table1, column_names)
table2a = load_and_clean_table(file, "Table 2A", columns_indices_table2, column_names)
table2c = load_and_clean_table(file, "Table 2C", columns_indices_table2, column_names)
table2d = load_and_clean_table(file, "Table 2D", columns_indices_table2, column_names)
table3a = load_and_clean_table(file, "Table 3A", columns_indices_table3, column_names)
table3b = load_and_clean_table(file, "Table 3B", columns_indices_table3, column_names)
table3d = load_and_clean_table(file, "Table 3D", columns_indices_table3, column_names)
table3e = load_and_clean_table(file, "Table 3E", columns_indices_table3, column_names)
table4 = load_and_clean_table(file, "Table 4", columns_indices_table4, column_names)

#print total len of each table
print(f"Table 1 length: {len(table1)}")
print(f"Table 2A length: {len(table2a)}")
print(f"Table 2C length: {len(table2c)}")
print(f"Table 2D length: {len(table2d)}")
print(f"Table 3A length: {len(table3a)}")
print(f"Table 3B length: {len(table3b)}")
print(f"Table 3D length: {len(table3d)}")
print(f"Table 3E length: {len(table3e)}")
print(f"Table 4 length: {len(table4)}")
print(f"Total length: {len(table1) + len(table2a) + len(table2c) + len(table2d) + len(table3a) + len(table3b) + len(table3d) + len(table3e) + len(table4)}")


# merge all tables into one DataFrame
merged_table = pd.concat([table1, table2a, table2c, table2d, table3a, table3b, table3d, table3e, table4], ignore_index=True)

Table 1 length: 15
Table 2A length: 31
Table 2C length: 28
Table 2D length: 25
Table 3A length: 33
Table 3B length: 36
Table 3D length: 36
Table 3E length: 36
Table 4 length: 2
Total length: 242


In [117]:
#bring the sheet edp
df = pd.read_excel(file, sheet_name="Edp")
#keep only from row 33
df = df.iloc[31:]
#drop last 5 columns
df = df.iloc[:, :-5]
#move last column to the front
df = df[[df.columns[-1]] + list(df.columns[:-1])]
#drop last 4 columns
df = df.iloc[:, :-5]
#make the first row as header
df.columns = df.iloc[0]
#drop the first row
df = df[1:]
#reset index
df.reset_index(drop=True, inplace=True)
#in column REF_AREA make all values "EL"
#df["REF_AREA"] = "EL"
df

31,SDMX series,FREQ,ADJUSTMENT,REF_AREA,COUNTERPART_AREA,REF_SECTOR,COUNTERPART_SECTOR,CONSOLIDATION,ACCOUNTING_ENTRY,STO,INSTR_ASSET,MATURITY,VALUATION,PRICES,CURRENCY_DENOM,CUST_BREAKDOWN,UNIT_MEASURE,TRANSFORMATION,CL_NA_TABLEID/DATES
0,A.N.@@._Z.S13._Z._Z.B.B9._Z._Z.S.V._T._T.XDC.N...,A,N,@@,_Z,S13,_Z,_Z,B,B9,_Z,_Z,S,V,_T,_T,XDC,N,EDP1
1,A.N.@@._Z.S1311._Z._Z.B.B9._Z._Z.S.V._T._T.XDC...,A,N,@@,_Z,S1311,_Z,_Z,B,B9,_Z,_Z,S,V,_T,_T,XDC,N,EDP1
2,A.N.@@._Z.S1312._Z._Z.B.B9._Z._Z.S.V._T._T.XDC...,A,N,@@,_Z,S1312,_Z,_Z,B,B9,_Z,_Z,S,V,_T,_T,XDC,N,EDP1
3,A.N.@@._Z.S1313._Z._Z.B.B9._Z._Z.S.V._T._T.XDC...,A,N,@@,_Z,S1313,_Z,_Z,B,B9,_Z,_Z,S,V,_T,_T,XDC,N,EDP1
4,A.N.@@._Z.S1314._Z._Z.B.B9._Z._Z.S.V._T._T.XDC...,A,N,@@,_Z,S1314,_Z,_Z,B,B9,_Z,_Z,S,V,_T,_T,XDC,N,EDP1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307,A.N.@@._Z.S1314._Z.C.L.LE.GD.T.F.V._T._T.XDC.N...,A,N,@@,_Z,S1314,_Z,C,L,LE,GD,T,F,V,_T,_T,XDC,N,EDP3
308,A.N.@@._Z.S1314.S13T._Z.A.LE.GD.T.F.V._T._T.XD...,A,N,@@,_Z,S1314,S13T,_Z,A,LE,GD,T,F,V,_T,_T,XDC,N,EDP3
309,A.N.@@._Z.S13._Z.C.L.F.F81.T.S.V._T._T.XDC.N.EDP4,A,N,@@,_Z,S13,_Z,C,L,F,F81,T,S,V,_T,_T,XDC,N,EDP4
310,A.N.@@._Z.S13._Z.C.L.LE.FPU.T.S.V._T._T.XDC.N....,A,N,@@,_Z,S13,_Z,C,L,LE,FPU,T,S,V,_T,_T,XDC,N,EDP4


In [118]:
merged_table

,Code,2021,2022,2023,2024
0,A.N.@@._Z.S13._Z._Z.B.B9._Z._Z._Z.XDC._T.S.V.N...,-13044,-5104,-3042,3181
1,A.N.@@._Z.S1311._Z._Z.B.B9._Z._Z._Z.XDC._T.S.V...,-14483,-7497,-3380,2050
2,A.N.@@._Z.S1313._Z._Z.B.B9._Z._Z._Z.XDC._T.S.V...,-161,-350,-599,30
3,A.N.@@._Z.S1314._Z._Z.B.B9._Z._Z._Z.XDC._T.S.V...,1600,2743,937,1101
4,A.N.@@._Z.S13._Z.C.L.LE.GD.T._Z.XDC._T.F.V.N._...,364141,368005,369110,364885
...,...,...,...,...,...
237,A.N.@@._Z.S1314.S13.C.NE.LE.GD.T._Z.XDC._T.F.V...,-17811,-22023,-22972.0,-21901
238,A.N.@@._Z.S1314._Z.C.L.LE.GD.T._Z.XDC._T.F.V.N...,106,98,91.0,83
239,A.N.@@._Z.S1314.S13T._Z.A.LE.GD.T._Z.XDC._T.F....,17917,22121,23063.0,21984
240,A.N.@@._Z.S13._Z.C.L.F.F81.T._Z.XDC._T.S.V.N._...,2701,3282,4405,4307


In [119]:
# df print first cell
print(df.iloc[0, 0])  # Print the first cell of the merged table



merged_table["Code_adjusted"] = merged_table.iloc[:, 0].apply(custom_reorder)

# check how many code_adjusted are in not in df
not_in_df = merged_table[~merged_table["Code_adjusted"].isin(df.iloc[:, 0])]
print(f"Codes in merged_table not in df: {len(not_in_df)}")

# print which codes are not in df
print("Codes not in df:")
print(not_in_df["Code_adjusted"].unique())

A.N.@@._Z.S13._Z._Z.B.B9._Z._Z.S.V._T._T.XDC.N.EDP1
Codes in merged_table not in df: 0
Codes not in df:
[]


In [125]:
# now we can merge the two DataFrames on the adjusted code
merged_df = pd.merge(merged_table, df, left_on="Code_adjusted", right_on=df.columns[0], how="left", suffixes=('', '_edp'))
# remove column index 0, 5, 6
merged_df = merged_df.drop(columns=[merged_df.columns[0], merged_df.columns[5], merged_df.columns[6]])
# ref_area make all values "EL"
merged_df["REF_AREA"] = "EL"

In [126]:
merged_df

,2021,2022,2023,2024,FREQ,ADJUSTMENT,REF_AREA,COUNTERPART_AREA,REF_SECTOR,COUNTERPART_SECTOR,...,STO,INSTR_ASSET,MATURITY,VALUATION,PRICES,CURRENCY_DENOM,CUST_BREAKDOWN,UNIT_MEASURE,TRANSFORMATION,CL_NA_TABLEID/DATES
0,-13044,-5104,-3042,3181,A,N,EL,_Z,S13,_Z,...,B9,_Z,_Z,S,V,_T,_T,XDC,N,EDP1
1,-14483,-7497,-3380,2050,A,N,EL,_Z,S1311,_Z,...,B9,_Z,_Z,S,V,_T,_T,XDC,N,EDP1
2,-161,-350,-599,30,A,N,EL,_Z,S1313,_Z,...,B9,_Z,_Z,S,V,_T,_T,XDC,N,EDP1
3,1600,2743,937,1101,A,N,EL,_Z,S1314,_Z,...,B9,_Z,_Z,S,V,_T,_T,XDC,N,EDP1
4,364141,368005,369110,364885,A,N,EL,_Z,S13,_Z,...,LE,GD,T,F,V,_T,_T,XDC,N,EDP1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237,-17811,-22023,-22972.0,-21901,A,N,EL,_Z,S1314,S13,...,LE,GD,T,F,V,_T,_T,XDC,N,EDP3
238,106,98,91.0,83,A,N,EL,_Z,S1314,_Z,...,LE,GD,T,F,V,_T,_T,XDC,N,EDP3
239,17917,22121,23063.0,21984,A,N,EL,_Z,S1314,S13T,...,LE,GD,T,F,V,_T,_T,XDC,N,EDP3
240,2701,3282,4405,4307,A,N,EL,_Z,S13,_Z,...,F,F81,T,S,V,_T,_T,XDC,N,EDP4


In [127]:
#save to xlsx
output_file = "assets/sel03/merged_table.xlsx"
merged_df.to_excel(output_file, index=False)